# Object Localization Demonstration Notebook
### Import statements

In [2]:
import pyrealsense2 as rs
import cv2
import detect
import numpy as np
import time
import torch
import io

ModuleNotFoundError: No module named 'pyrealsense2'

### Declaring constant variables, obtained from values in Intel and VRC documentation

In [3]:
focal_length = ((448.0-172.0) * 24.0) / 11.0
# ['Blue Goal', 'Blue Platform', 'Blue Robot', 'Neutral Goal', 'Red Goal', 'Red Platform', 'Red Robot', 'Ring']
width_dict = {"7":3.5, "3":12.5, "0":12.5, "4":12.5, "2":5.5, "5":5.5, "1":53.0, "6":53.0}

### Starting image pipeline from D435 camera

In [6]:
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
pipeline.start(config)

### Object localization method

In [7]:
def obj_distance(cls, x, y, w, h, depth_frame):
    # calculating distance using trigonometric properties
    trig_distance = (width_dict[str(cls)] * focal_length)/w
    
    # extract average distance from depth map and convert to inches
    depth_distance_meters = (depth_frame.get_distance(x, y) +\
                             depth_frame.get_distance(x+2, y) +\
                             depth_frame.get_distance(x, y+2) +\
                             depth_frame.get_distance(x-2, y) +\
                             depth_frame.get_distance(x, y-2))/5.0
    depth_distance = 39.3701 * depth_distance_meters
    
    # weighting and combining localization methods
    distance = (trig_distance * .2) + (depth_distance_meters * .8) 
    
    # in the event that depthmap can't detect distance, only use trig distance
    if (depth_distance_meters == 0):
        distance = trig_distance
    
    return distance

### Localization Fusion Method

In [ ]:
cam_height = 10
cam_angle = -5

In [ ]:
def obj_lf(obj_x, obj_y, obj_distance, robot_coords):
    
    
    
    
    return (calculated_x, calculated_y)

In [8]:
# Demo x, y, theta
robot_coords = (30, 30, 90)

### Using identification and localization to process images from the pipeline

In [ ]:
while True:
    # extracting data from the image pipeline
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()
    color_image = np.asanyarray(color_frame.get_data())

    # IO for trained YOLOv5 model (color_img->game_objects)
    cv2.imwrite('img.png', color_image)

    !python detect.py --source img.png --weights static/best_torchscript.pt --conf 0.45 --name yolo_obj --view-img --save-txt --nosave --exist-ok

    # reading labels from localization output
    game_objects = []
    with open('runs/detect/yolo_obj/labels/img.txt', 'r+') as f:
        game_objects = f.readlines()
        f.truncate(0)
        f.close()

    # calculating distance for all game objects in frame
    for obj in game_objects:
        obj_array = obj.split()
        cls, x, y, w, h = int(obj_array[0]), int(round(float(obj_array[1]))), int(round(float(obj_array[2]))), float(obj_array[3]), float(obj_array[4])
#         print(f'{obj.split()[0]}: {obj_distance(obj, depth_frame)}')
        obj_distance = obj_distance(cls, x, y, w, h, depth_frame)
        obj_location = obj_lf(x, y, obj_distance, robot_coords)

detect: weights=['static/best_torchscript.pt'], source=img.png, imgsz=[640, 640], conf_thres=0.45, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_obj, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  d8f4ccd torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/1 C:\Users\wellerj\Documents\github\senior-design-cv\localization\img.png: 640x640 Done. (1.709s)
Speed: 10.3ms pre-process, 1709.2ms inference, 1.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\yolo_obj
101 labels saved to runs\detect\yolo_obj\labels


3: 21.436225156173514


detect: weights=['static/best_torchscript.pt'], source=img.png, imgsz=[640, 640], conf_thres=0.45, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_obj, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  d8f4ccd torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/1 C:\Users\wellerj\Documents\github\senior-design-cv\localization\img.png: 640x640 1 class3, Done. (1.356s)
Speed: 0.0ms pre-process, 1356.0ms inference, 49.6ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\yolo_obj
101 labels saved to runs\detect\yolo_obj\labels


5: 8.414897786246405
7: 22.436205949646435
7: 18.70843643535267
3: 22.71331083546177
7: 13.64565396085163
7: 12.631620980915539


detect: weights=['static/best_torchscript.pt'], source=img.png, imgsz=[640, 640], conf_thres=0.45, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_obj, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  d8f4ccd torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/1 C:\Users\wellerj\Documents\github\senior-design-cv\localization\img.png: 640x640 1 class3, 1 class5, 4 class7s, Done. (1.376s)
Speed: 0.0ms pre-process, 1376.1ms inference, 38.5ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\yolo_obj
101 labels saved to runs\detect\yolo_obj\labels


...

detect: weights=['static/best_torchscript.pt'], source=img.png, imgsz=[640, 640], conf_thres=0.45, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_obj, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  d8f4ccd torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/1 C:\Users\wellerj\Documents\github\senior-design-cv\localization\img.png: 640x640 5 class2s, 1 class4, 1 class7, Done. (1.385s)
Speed: 2.7ms pre-process, 1384.9ms inference, 7.4ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\yolo_obj
101 labels saved to runs\detect\yolo_obj\labels


2: 41.60640001296997
2: 37.00240001678466
2: 24.733973344167072
3: 27.086316889601868
0: 29.463848976988892
2: 236.57142857142853
2: 35.06267790091665
0: 22.66643083343998


detect: weights=['static/best_torchscript.pt'], source=img.png, imgsz=[640, 640], conf_thres=0.45, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_obj, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  d8f4ccd torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/1 C:\Users\wellerj\Documents\github\senior-design-cv\localization\img.png: 640x640 2 class0s, 5 class2s, 1 class3, Done. (1.506s)
Speed: 6.3ms pre-process, 1506.4ms inference, 12.8ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\yolo_obj
101 labels saved to runs\detect\yolo_obj\labels


4: 29.788156589518255
0: 32.286787791040005
2: 30.370370921221646
2: 51.21544617139376
7: 38.59010116900294


detect: weights=['static/best_torchscript.pt'], source=img.png, imgsz=[640, 640], conf_thres=0.45, iou_thres=0.45, max_det=1000, device=, view_img=True, save_txt=True, save_conf=False, save_crop=False, nosave=True, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=runs\detect, name=yolo_obj, exist_ok=True, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  d8f4ccd torch 1.10.0 CPU

Loading static/best_torchscript.pt for TorchScript inference...
image 1/1 C:\Users\wellerj\Documents\github\senior-design-cv\localization\img.png: 640x640 1 class0, 2 class2s, 1 class4, 1 class7, Done. (1.340s)
Speed: 2.9ms pre-process, 1339.5ms inference, 3.0ms NMS per image at shape (1, 3, 640, 640)
Results saved to runs\detect\yolo_obj
101 labels saved to runs\detect\yolo_obj\labels
